<h1><center>TECHNOLOGY</center></h1>

In [461]:
import requests
import json
from pprint import pprint
from datetime import datetime as dt
from bs4 import BeautifulSoup
import datetime
import string
import pandas as pd
import numpy as np
from collections import Counter
import pickle

### NLTK imports
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

### Gensim imports
import gensim
from gensim import corpora

### SKLEARN imports
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# ignoring warnings
import warnings
warnings.filterwarnings('ignore')

<h1><center>DATA EXTRACTION</center></h1>

In [124]:
# Pulling data for Category : Technology
# r = requests.get('https://a.4cdn.org/g/catalog.json')
# r = r.json()

# pprint(r)

In [54]:
#save into a JSON file
# import json
# import re
# with open('Tech.json', 'w') as fout:
#     json.dump(r , fout)

In [449]:
# Read from a JSON file
with open(r"Tech.json", "r") as read_file:
    data = json.load(read_file)

In [450]:
# More exploration
# Pulling the comments and replies to the comments from the JSON file
comments = []
date_time = []
for i in range(len(data)):
    for j in range(len(data[i]['threads'])):
        if 'com' in data[i]['threads'][j].keys() :
            one_comm = data[i]['threads'][j]['com']
            date_time += [datetime.datetime.fromtimestamp(data[i]['threads'][j]['time']).strftime("%B %d, %Y")]
            soup1 = BeautifulSoup(one_comm)
            comments += [soup1.get_text()]
            if 'last_replies' in data[i]['threads'][j].keys():
                for k in range(len(data[i]['threads'][j]['last_replies'])):
                    if 'com' in data[i]['threads'][j]['last_replies'][k]:
                        reply = data[i]['threads'][j]['last_replies'][k]['com']
                        date_time += [datetime.datetime.fromtimestamp(data[i]['threads'][j]['last_replies'][k]['time']).strftime("%B %d, %Y")]
                        soup2 = BeautifulSoup(reply)
                        comments += [soup2.get_text()]  

In [451]:
print(date_time[0])
print(comments[0])

December 20, 2015
The /g/ Wiki:http://wiki.installgentoo.com/

/g/ is for the discussion of technology and related topics.
/g/ is NOT your personal tech support team or personal consumer review site.
For tech support/issues with computers, use /wsr/ - Worksafe Requests or one of the following:
https://startpage.com/ or https://duckduckgo.com (i.e., fucking google it)
https://stackexchange.com/
http://www.logicalincrements.com/

You can also search the catalog for a specific term by using:
https://boards.4chan.org/g/searchword or by clicking on [Search] 

Always check the catalog before creating a thread:>>>/g/catalog

Please check the rules before you post:
Begging for cryptocurrency is against the rules!

To use the Code tag, book-end your body of code with: [code] and [/code]


In [452]:
# The first comment is from Dec 2015 and is the guidelines for using the forum. Removing it for the current data analysis.
comments.pop(0)
date_time.pop(0)

'December 20, 2015'

In [453]:
comments[0]

"https://www.guru3d.com/news-story/ryzen-4000-zen-3-desktop-cpu-compatibility-with-x570-b550not-400-or-lower.html> Earlier and cheaper motherboards use smaller bios (16MB) chips, that's less storage space than needed. As a result, it became increasingly difficult for motherboard manufacturers to continue to add support for newer processors; it simply no longer fits on the bios chips.  AMD will need 32MB BIOS ROM chipsLike why can’t they have a separate bios update that adds support zen3 that has older CPU support removed?"

### Observations:
1. We have extracted 772 comments from the technology section of the website.
2. The comments were made on May 7th, 2020 and May 8th, 2020

<h1><center>DATA PREPARATION/CLEANING</center></h1>

### 1.      CONVERTS WORDS TO THEIR BASE FORMS

### 2. DATA CLEANING

#### The next step is to clean the data and remove things which do not add meaning to our analysis. Broadly, we will be looking at removing punctuations and stop words 

In [357]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
def clean_data(tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tokens):
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token)
    return cleaned_tokens


In [454]:
cleaned_data = []
with open("stop_words.txt", "r") as input:
        extra_stop_words = [line.split(",")[0] for line in input.read().splitlines()]
stop_words = stopwords.words('english')
newStopWords = ['/g/','n\'t','\'s','\'\'','``','would','get','like','use','one','\'m','http','n','0', 'thus','x','1','say','good','much','want','go','run','need','new','even','shit','fuck']
stop_words+=newStopWords
print(len(stop_words))
stop_words+=extra_stop_words
print(len(stop_words))
for text in comments:
    cleaned_data += [clean_data(word_tokenize(text), stop_words)]

207
398


<h1><center>DATA EXPLORATION</center></h1>

### Method : Topic Modelling using Latent Dirichlet Allocation
#### Create a dictionary from data , then convert to bag-of-words corpus and save the dictionary and corpus for future use

In [455]:
dictionary = corpora.Dictionary(cleaned_data)
corpus = [dictionary.doc2bow(text) for text in cleaned_data]

In [456]:
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

### Let's fit the LDA model. But, why LDA?
LDA would provide us good composition of topic-keyword distribution
Main Idea : Each document can be described by a distribution of topics and each topic can be described by a distribution of words

In [457]:
NUM_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model_Tech.gensim')

In [458]:
topics = ldamodel.print_topics(num_words=6)
for topic in topics:
    print(topic)

(0, '0.004*"time" + 0.003*"could" + 0.003*"power" + 0.003*"think" + 0.003*"battery" + 0.003*"year"')
(1, '0.003*"people" + 0.003*"’" + 0.002*"well" + 0.002*"software" + 0.002*"support" + 0.002*"big"')
(2, '0.004*"old" + 0.003*"phone" + 0.003*"’" + 0.003*"buy" + 0.003*"Windows" + 0.003*"support"')


### Topic 0 includes words like "power", "battery", "year",. This sounds like a topic related to some battery backup and power discussion
### Topic 0 includes words like "software", ''support". This is a topic related to some software and support discussion
### Topic 1 includes words like "phone", "Windows", "old". This sounds like a topic related to different operating system and buying a phone
    

<h1><center>VISUALIZE THE TOPIC KEYWORDS</center></h1>

#### Saliency : a measure how mcuh the term tells you about the topic
#### Relevance : A weighted average of the probability of the word given the topic and the word given the topic normalized by the probability of the topic
#### The size of the bubble shows the importance of the topics relative to the data

#### We can view the frequency of the top words in a given topic by howevering over the topic

In [459]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model_Tech.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

Inference : 
    1. Topic 1,2 and 3 are far apart in intertopic distance map showing the large semantic differences between the topics
    2. The topic 1 looks related to computer and phone for university use
    3. The main topics for discussion in Topic 1 are the operating systems - linus and Windows and the LTSC for windows
    4. Topic 2 discusses Software, support and performance
    4. Topic 3 discussed phone, windows, games and monitor